In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision.transforms as tr 

from dataset import make_dataset, collate_fn

In [40]:
import sys
import os

sys.path.append(os.path.abspath('..'))

from data.dataset import make_dataset, collate_fn
from torch.utils.data import DataLoader

# 설정 변수
batch_size = 4
size = 512

image_dir_train = 'train/images/'
bbox_dir_train = 'train/bbox/'

image_dir_val = 'validation/images/'
bbox_dir_val = 'validation/bbox/'

image_dir_test = 'test/images/'
bbox_dir_test = 'test/bbox/'


# 데이터셋 생성
train_set = make_dataset(
    image_dir=image_dir_train,
    bbox_dir=bbox_dir_train
)

val_set = make_dataset(
    image_dir=image_dir_val,
    bbox_dir=bbox_dir_val
)

test_set = make_dataset(
    image_dir=image_dir_test,
    bbox_dir=bbox_dir_test
)

# DataLoader 생성
train_loader = DataLoader(
    train_set, 
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
    collate_fn=collate_fn,
    num_workers=0
)

val_loader = DataLoader(
    val_set, 
    batch_size=batch_size,
    shuffle=False,
    pin_memory=True,
    collate_fn=collate_fn,
    num_workers=0
)

test_loader = DataLoader(
    test_set, 
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn
)

# 데이터 로드 및 출력
images, labels, bbox = next(iter(val_loader))

print(images.shape)
print(labels)
print(bbox)

torch.Size([4, 3, 512, 512])
(tensor([4]), tensor([4]), tensor([4]), tensor([4]))
(tensor([[ 200.9026,    0.0000, 1158.6770,  960.9201]]), tensor([[ 266.9031,  348.4540, 1536.1183,  878.4917]]), tensor([[ 897.0312,    0.0000, 1280.0000,  720.0000]]), tensor([[ 348.3173,  163.5216, 1481.7118,  965.0387]]))
